<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Yolo2Rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import random
import shutil

# Rutas de las carpetas de imágenes y etiquetas originales
images_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/images"
labels_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/labels"

# Ruta principal del dataset final
main_dir = "/content/dataset_MaskRCNN"

# Subcarpetas del dataset final
train_dir = os.path.join(main_dir, "train")
test_dir = os.path.join(main_dir, "test")
val_dir = os.path.join(main_dir, "val")

# Rutas de las carpetas de train, test y val para imágenes y etiquetas
train_images_dir = os.path.join(train_dir, "images")
train_labels_dir = os.path.join(train_dir, "labels")
test_images_dir = os.path.join(test_dir, "images")
test_labels_dir = os.path.join(test_dir, "labels")
val_images_dir = os.path.join(val_dir, "images")
val_labels_dir = os.path.join(val_dir, "labels")

# Función para crear carpetas si no existen
def create_directories_if_not_exist(*directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Crear carpetas de train, test y val si no existen
create_directories_if_not_exist(train_dir, test_dir, val_dir)
create_directories_if_not_exist(train_images_dir, train_labels_dir, test_images_dir, test_labels_dir, val_images_dir, val_labels_dir)

# Proporciones para train, test y val
train_percent = 0.6
test_percent = 0.2
val_percent = 0.2

# Obtener lista de nombres de archivos de imágenes
image_files = os.listdir(images_dir)
# Filtrar solo los archivos con extensión .jpg
image_files = [file for file in image_files if file.endswith(".jpg")]

# Barajar la lista de nombres de archivos
random.shuffle(image_files)

# Calcular el número de archivos para cada conjunto
total_images = len(image_files)
num_train = int(total_images * train_percent)
num_test = int(total_images * test_percent)
num_val = total_images - num_train - num_test

# Dividir la lista de nombres de archivos en train, test y val
train_images = image_files[:num_train]
test_images = image_files[num_train:num_train+num_test]
val_images = image_files[num_train+num_test:]

# Crear carpetas de train, test y val si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Copiar imágenes y etiquetas a las carpetas correspondientes
def copy_files(files, source_dir, dest_images_dir, dest_labels_dir):
    for file in files:
        # Copiar imágenes
        shutil.copy(os.path.join(source_dir, file), dest_images_dir)
        # Copiar etiquetas
        label_file = os.path.splitext(file)[0] + ".txt"
        shutil.copy(os.path.join(labels_dir, label_file), dest_labels_dir)

copy_files(train_images, images_dir, train_images_dir, train_labels_dir)
copy_files(test_images, images_dir, test_images_dir, test_labels_dir)
copy_files(val_images, images_dir, val_images_dir, val_labels_dir)

print("División de imágenes y etiquetas completada.")


División de imágenes y etiquetas completada.


In [8]:
# Función para leer archivos .txt de etiquetas de YOLO
def read_yolo_labels(yolo_label_dir):
    yolo_labels = {}
    for file_name in os.listdir(yolo_label_dir):
        if file_name.endswith(".txt"):
            with open(os.path.join(yolo_label_dir, file_name), "r") as f:
                content = f.read().strip().split("\n")
                labels = []
                for line in content:
                    parts = line.split(" ")
                    class_id = int(parts[0])
                    points = []
                    for i in range(1, len(parts), 2):
                        x, y = map(float, parts[i:i+2])
                        point = [x, y]
                        points.append(point)
                    label = {"class_id": class_id, "points": points}
                    labels.append(label)
                yolo_labels[file_name] = labels
    return yolo_labels

# Leer archivos de etiquetas de YOLO para train, test y val
train_yolo_labels = read_yolo_labels(train_labels_dir)
test_yolo_labels = read_yolo_labels(test_labels_dir)
val_yolo_labels = read_yolo_labels(val_labels_dir)

print("Etiquetas de YOLO leídas para train, test y val.")


Etiquetas de YOLO leídas para train, test y val.


In [13]:
print(train_yolo_labels)

{'100H0164.txt': [{'class_id': 0, 'points': [[0.28259259259259273, 0.056365740740740626], [0.3880925925925888, 0.03912037037037025], [0.4483703703703666, 0.04826388888888878], [0.49559259259258387, 0.06111111111111099], [0.5197592592592506, 0.08067129629629619], [0.5817037037036937, 0.12685185185185174], [0.5983703703703603, 0.15706018518518508], [0.6140740740740627, 0.18831018518518508], [0.6262037037036924, 0.22789351851851838], [0.6196296296296182, 0.2576388888888888], [0.6270370370370257, 0.29479166666666656], [0.6197222222222109, 0.3134259259259259], [0.5992592592592478, 0.3133101851851851], [0.5622222222222107, 0.3064814814814814], [0.5390740740740626, 0.3030092592592592], [0.5131111111111035, 0.29837962962962955], [0.5029259259259183, 0.2797453703703703], [0.48255555555554797, 0.22872685185185163], [0.46588888888888125, 0.19550925925925905], [0.4390740740740741, 0.18495370370370362], [0.3936111111111112, 0.17569444444444435], [0.3362037037037038, 0.18726851851851845], [0.2853703

In [15]:
def convert_to_mask_rcnn_labels(yolo_labels):
    mask_rcnn_labels = {}
    for file_name, yolo_objs in yolo_labels.items():
        mask_rcnn_labels[file_name.replace(".txt", ".jpg")] = {
            "fileref": "",
            "size": 123456,  # Tamaño de la imagen (ajustar según sea necesario)
            "filename": file_name.replace(".txt", ".jpg"),
            "base64_img_data": "",
            "file_attributes": {},
            "regions": {}
        }
        for i, obj in enumerate(yolo_objs):
            mask_rcnn_labels[file_name.replace(".txt", ".jpg")]["regions"][str(i)] = {
                "shape_attributes": {
                    "name": "polygon",
                    "all_points_x": [int(point[0] * 123) for point in obj["points"]],  # Ajustar a las dimensiones de la imagen
                    "all_points_y": [int(point[1] * 456) for point in obj["points"]]  # Ajustar a las dimensiones de la imagen
                },
                "region_attributes": {
                    "class": obj["class_id"]  # Clase del objeto
                }
            }
    return mask_rcnn_labels


In [16]:
import json

# Carpeta de train
train_mask_rcnn_labels = convert_to_mask_rcnn_labels(train_yolo_labels)
train_RCNN_labels_dir = "/content/dataset_MaskRCNN/train/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(train_RCNN_labels_dir):
    os.makedirs(train_RCNN_labels_dir)

for file_name, label_data in train_mask_rcnn_labels.items():
    with open(os.path.join(train_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)

# Carpeta de test
test_mask_rcnn_labels = convert_to_mask_rcnn_labels(test_yolo_labels)
test_RCNN_labels_dir = "/content/dataset_MaskRCNN/test/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(test_RCNN_labels_dir):
    os.makedirs(test_RCNN_labels_dir)

for file_name, label_data in test_mask_rcnn_labels.items():
    with open(os.path.join(test_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)

# Carpeta de val
val_mask_rcnn_labels = convert_to_mask_rcnn_labels(val_yolo_labels)
val_RCNN_labels_dir = "/content/dataset_MaskRCNN/val/RCNN_labels"

# Guardar los resultados en la carpeta correspondiente
if not os.path.exists(val_RCNN_labels_dir):
    os.makedirs(val_RCNN_labels_dir)

for file_name, label_data in val_mask_rcnn_labels.items():
    with open(os.path.join(val_RCNN_labels_dir, file_name.replace(".jpg", ".json")), "w") as f:
        json.dump(label_data, f)


In [5]:
# PRUEBA DE LECTURA DE LAS EQIQUETAS DE YOLO

# Leer etiquetas de YOLOv9 para una sola imagen
def read_yolo_labels(yolo_label_file):
    labels = []
    with open(yolo_label_file, "r") as f:
        content = f.read().strip().split("\n")
        for line in content:
            parts = line.split(" ")
            class_id = int(parts[0])
            points = []
            for i in range(1, len(parts), 2):
                x, y = map(float, parts[i:i+2])
                point = [x, y]
                points.append(point)
            label = {"class_id": class_id, "points": points}
            labels.append(label)
    return labels

# Ruta del archivo de etiquetas de YOLOv9 para una sola imagen
yolo_label_file = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/labels/100H0060.txt"

# Obtener las etiquetas de YOLOv9 para una sola imagen
yolo_labels = read_yolo_labels(yolo_label_file)

# Imprimir las etiquetas de YOLOv9 para una sola imagen
print("Etiquetas de YOLOv9 para una sola imagen:")
for label in yolo_labels:
    print("Clase ID:", label["class_id"])
    print("Puntos de segmentación:")
    for point in label["points"]:
        print("    ", point)


Etiquetas de YOLOv9 para una sola imagen:
Clase ID: 0
Puntos de segmentación:
     [0.6861111111111112, 0.751388888888889]
     [0.7027777777777777, 0.7290509259259259]
     [0.712037037037037, 0.7278935185185185]
     [0.7419444444444445, 0.7417824074074074]
     [0.7425925925925926, 0.7848611111111095]
     [0.7186111111111111, 0.8114814814814798]
     [0.6906481481481481, 0.820740740740739]
     [0.6722222222222223, 0.8159953703703686]
     [0.674074074074074, 0.7788425925925908]
Clase ID: 0
Puntos de segmentación:
     [0.6157407407407407, 0.026736111111111138]
     [0.6342592592592593, 0.011574074074074073]
     [0.669537037037037, 0.005787037037037037]
     [0.7342592592592565, 0.010532407407407433]
     [0.7694444444444418, 0.021874999999999974]
     [0.8001111111111067, 0.013773148148148121]
     [0.8369629629629586, 0.02546296296296296]
     [0.8546851851851793, 0.048726851851851875]
     [0.9037962962962925, 0.1423611111111111]
     [0.9592407407407494, 0.1983101851851849]
  